## Getting a bit more deeper into Agents
- Conversational Memory
- ReAct DocStore Agent
- Self-ask with Search Agent

#### Install the following Libraries if they are not installed already

In [3]:
#!pip install langchain
#!pip install langchain-community
#!pip install langchain-openai
#!pip install python-dotenv
#!pip install langchain-google-genai
#!pip install langchainhub
#!pip install wikipedia
#!pip install -U duckduckgo-search

#### Import Libraries

In [57]:
from langchain.agents import create_react_agent, load_tools, create_structured_chat_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.chains import LLMMathChain
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent, create_self_ask_with_search_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
import warnings
import os

In [7]:
load_dotenv()

True

#### Set your LLM

## Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [97]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=0)

#### Load the LLMMath tool

In [105]:
problem_chain = LLMMathChain.from_llm(llm=llm)
math_tool = Tool.from_function(name="Calculator",
                func=problem_chain.run,
                 description="""Useful for when you need to answer questions 
                        about math. This tool is only for math questions and nothing else. Only input
                        math expressions.""")

tools = [math_tool]

#### Instantiate Conversational memory for our Agent

In [108]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

### Establish the System and Human Prompts and create the Prompt object 

In [111]:
# Define the Prompt Template

system = '''Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input arithmetic question to answer containing numeric digits
Thought: consider previous steps if any
Action:
```
$JSON_BLOB
```
Observation: action result of the arithmetic operation
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'''

human = '''

{input}

{agent_scratchpad}

(reminder to respond in a JSON blob no matter what)'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
    ]
)


### Create the Agent and Agent Executor Objects

In [114]:
conversational_agent = create_structured_chat_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=conversational_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
    max_iterations=10,
)

### Set up a Loop if accepting user input (until "X" > Enter is pressed).
<b>We try with a sequence of arithmetic questions:</b><br>
Sample:<br>
User:  Add 5 to 8<br>
Agent: The sum of 5 and 8 is 13.<br>
User:  add 6 to it<br>
Agent: The result of adding 6 to 13 is 19.<br>
User:  multiply this now with 5<br>
Agent: The result of multiplying 19 by 5 is 95.<br>

In [119]:
user_input = ""
while user_input != "X":  # terminate the input loop by typing X (capital "X")
    user_input = input("User: ")
    chat_history = memory.buffer_as_messages
    response = agent_executor.invoke({
        "input": user_input,
        "chat_history": chat_history,
    })
    print("Agent:", response['output'])


User:  Add 5 to 7




> Entering new AgentExecutor chain...
{
  "action": "Calculator",
  "action_input": {
    "tool_input": "5 + 7"
  }
}Answer: 12{
  "action": "Final Answer",
  "action_input": "12"
}

> Finished chain.
Agent: 12


User:  Multiply the result by 6




> Entering new AgentExecutor chain...
{
  "action": "Calculator",
  "action_input": {
    "tool_input": "12 * 6"
  }
}Answer: 72{
  "action": "Final Answer",
  "action_input": "72"
}

> Finished chain.
Agent: 72


User:  Subtract 4 from it




> Entering new AgentExecutor chain...
{
  "action": "Calculator",
  "action_input": {
    "tool_input": "72 - 4"
  }
}Answer: 68{
  "action": "Final Answer",
  "action_input": "68"
}

> Finished chain.
Agent: 68


User:  X




> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "68"
}

> Finished chain.
Agent: 68


# ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [123]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

### Establish the Tools (Wikipedia search and Lookup)

In [152]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

### Set up the Prompts and intantiate the Prompt object using Prompt Template

In [155]:
template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

### Create the Agent and Agent Executor

In [158]:
docstore_agent = create_react_agent(
    tools=tools,
    llm=llm,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=docstore_agent,
    tools=tools,
    #verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

### Invoke the Agent Executor

In [161]:
response = agent_executor.invoke(
    {"input": "Tell me a few key things about Martin Luther King."})

In [162]:
print(response)

{'input': 'Tell me a few key things about Martin Luther King.', 'output': 'Martin Luther King Jr. was a prominent leader in the civil rights movement, known for his use of nonviolent resistance against discrimination. He led marches, organized protests, and advocated for civil rights legislation. He won the Nobel Peace Prize in 1964 and was assassinated in 1968. His legacy is honored with a national holiday, memorials, and streets named in his honor.', 'intermediate_steps': [(AgentAction(tool='Search', tool_input='Martin Luther King', log='I should search for information about Martin Luther King to gather key facts about him.\nAction: Search\nAction Input: Martin Luther King'), 'Martin Luther King Jr. (born Michael King Jr.; January 15, 1929 – April 4, 1968) was an American Baptist minister, activist, and political philosopher who was one of the most prominent leaders in the civil rights movement from 1955 until his assassination in 1968. A black church leader and a son of early civil 

# Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.

In [166]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

### Set up the Prompt and establish the Tools (DuckDuckgo Search)

In [178]:
# Get the prompt to use - you can modify this to build your own prompt!
prompt = hub.pull("hwchase17/self-ask-with-search")

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]
prompt = hub.pull("hwchase17/self-ask-with-search")

### Create the Agent and Agent Executor

In [180]:
agent = create_self_ask_with_search_agent(
    tools=tools,
    llm=llm,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [183]:
response = agent_executor.invoke(
    {"input": "Who was the president of USA when the first Moom landing took place?"})



> Entering new AgentExecutor chain...
Could not parse output: No.Invalid or incomplete responseFollow up: When did the first Moon landing take place?Apollo 11, in July 1969, climaxed the step-by-step procedure with a lunar landing; on July 20 astronaut Neil Armstrong and then Edwin ("Buzz") Aldrin became the first humans to set foot on the Moon's surface. Apollo 12. Apollo 12 lifting off from John F. Kennedy Space Center, Cape Canaveral, Florida, November 14, 1969. Neil Armstrong was the first human to walk on the surface of the moon. He was an astronaut who flew on two space missions. The first was Gemini 8. The second was Apollo 11, which landed on the moon in 1969. Armstrong was also an engineer, a pilot and a college professor. Read More Neil Armstrong (1930-2012) was a U.S. astronaut who became the first human to walk on the moon on July 20, 1969, as part of the Apollo 11 mission. With the dress rehearsal completed during Apollo 10 in May 1969, only a few weeks remained until Ap

In [185]:
print(response)

{'input': 'Who was the president of USA when the first Moom landing took place?', 'output': 'Richard Nixon', 'intermediate_steps': [(AgentAction(tool='_Exception', tool_input='Invalid or incomplete response', log='Could not parse output: No.'), 'Invalid or incomplete response'), (AgentAction(tool='Intermediate Answer', tool_input='When did the first Moon landing take place?', log='Follow up: When did the first Moon landing take place?'), 'Apollo 11, in July 1969, climaxed the step-by-step procedure with a lunar landing; on July 20 astronaut Neil Armstrong and then Edwin ("Buzz") Aldrin became the first humans to set foot on the Moon\'s surface. Apollo 12. Apollo 12 lifting off from John F. Kennedy Space Center, Cape Canaveral, Florida, November 14, 1969. Neil Armstrong was the first human to walk on the surface of the moon. He was an astronaut who flew on two space missions. The first was Gemini 8. The second was Apollo 11, which landed on the moon in 1969. Armstrong was also an engine